# Roadmap
## [DONE] Create primitive model, predicting for only one library (A3)
## Create model for all libraries
Here we can:
1. for-loops to create datasets and train separate models (might be computationally expensive)
2. **multi-input: whole dataset + encoded library name => BEST COMPROMISE (faster than the others bc only one model trained and no need to model inter-dependencies => good if we get 5% mae)**
3. data of one library + data of all libraries as inputs => better prediction for one library that accounts for inter-dependencies, BUT extremely computationally expensive (more inputs and more models)

## Tweak model to improve its accuracy
## Create pipeline for receiving data in real-time, once per hour  
## 'Integrate' model into the website

### Imports

In [18]:
import tracemalloc
tracemalloc.start()

import sqlite3

import pandas as pd
import numpy as np

import time
from datetime import timedelta

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

import tensorflow as tf
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, GRU, Dense, Concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Mount drive
# from google.colab import drive
# drive.mount('/content/drive')

# Connect to the database
db_path = "../prisma/dev.db"
conn = sqlite3.connect(db_path)

# Load pre-trained RNN model???
# model = load_model("path_to_your_model.h5")

### Check that Google Colab takes the database accordingly

In [19]:
#!ls "/content/drive/MyDrive/BibScraperModel"


In [20]:
#cursor = conn.cursor()
#cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
#print(cursor.fetchall())

### Fetch the data function

In [21]:
def fetch_latest_data():
    query = f"""
        SELECT name, year, month, day, chunk, percentage
        FROM BibData
    """
    df = pd.read_sql(query, conn)
    return df

### DATA FETCHING IDEA: TAKE DIRECTLY ONLY THE DATA FROM THE LAST 1 month or so
storage optimization, time efficiency purposes  
model could (theoretically) still be good with a dataset of 1 month

In [22]:
df = fetch_latest_data()
df = df.drop_duplicates().reset_index(drop=True)
df

,name,year,month,day,chunk,percentage
0,Ausleihzentrum Schloss Westflügel,2024,6,2,109,0
1,Bibliotheks­bereich A3,2024,6,2,109,100
2,Bibliotheks­bereich A5,2024,6,2,109,100
3,Bibliotheks­bereich Schloss Ehrenhof,2024,6,2,109,100
4,Bibliotheks­bereich Schloss Schneckenhof,2024,6,2,109,91
...,...,...,...,...,...,...
137561,Ausleihzentrum Schloss Westflügel,2024,12,11,100,88
137562,Bibliotheks­bereich A3,2024,12,11,100,98
137563,Bibliotheks­bereich A5,2024,12,11,100,95
137564,Bibliotheks­bereich Schloss Ehrenhof,2024,12,11,100,100


### Preprocess data into a dictionary with encoded libraries and their occupancy percentages

In [23]:
# Sort the DataFrame by Library, chronologically
df = df.sort_values(by=['name', 'year', 'month', 'day', 'chunk']).reset_index(drop=True)

# One-hot encode the library names
library_names = df['name'].unique().reshape(-1, 1)
encoder = OneHotEncoder(sparse_output=False)
one_hot_keys = encoder.fit_transform(library_names)

# Create the dictionary with one-hot encoded keys
data_by_library = {
    tuple(one_hot): df[df['name'] == library].drop(columns=['name', 'year', 'month', 'day', 'chunk'])
    for one_hot, library in zip(one_hot_keys, library_names.flatten())
}

# Normalize percentage values with min max scaler
# Because of floating-point operation errors, we round
for library, data in data_by_library.items():
  scaler = MinMaxScaler()
  data['Occupancy'] = scaler.fit_transform(data['percentage'].values.reshape(-1, 1)).round(2)
  data = data.drop(columns=['percentage'])

  # Update the dictionary with the changed DataFrame
  data_by_library[library] = data

data_by_library

{(1.0,
  0.0,
  0.0,
  0.0,
  0.0):        Occupancy
 0           0.00
 1           0.00
 2           0.00
 3           0.00
 4           0.00
 ...          ...
 27500       1.00
 27501       0.93
 27502       0.96
 27503       0.91
 27504       0.88
 
 [27505 rows x 1 columns],
 (0.0,
  1.0,
  0.0,
  0.0,
  0.0):        Occupancy
 27505       0.28
 27506       0.25
 27507       0.25
 27508       0.24
 27509       0.23
 ...          ...
 55010       1.00
 55011       1.00
 55012       1.00
 55013       0.98
 55014       0.98
 
 [27510 rows x 1 columns],
 (0.0,
  0.0,
  1.0,
  0.0,
  0.0):        Occupancy
 55015       0.49
 55016       0.45
 55017       0.43
 55018       0.38
 55019       0.34
 ...          ...
 82527       1.00
 82528       1.00
 82529       1.00
 82530       0.97
 82531       0.95
 
 [27517 rows x 1 columns],
 (0.0,
  0.0,
  0.0,
  1.0,
  0.0):         Occupancy
 82532        0.28
 82533        0.24
 82534        0.22
 82535        0.20
 82536        0.18
 ...       

### Prepare data for training

In [24]:
# We look at 5 days in the past to predict the next value (5 days x 24 hours x 6 chunks = 720 chunks)
past = 720

# We want to predict one hour ahead
future = 6

# We sample data every hour - look at it every 6 chunks within the (past, future) timeframe
# We do this to reduce the amount of data to process to a manageable size
sampling_rate = 6

# Define the sequence length:
# We actually look at 720 / 6 = 120 timesteps in the past (120 points of past data)
sequence_length = int(past / sampling_rate)

# 80% train, 20% validation
# Note that there is no test data, since we do not actually know the future values to test against
split_fraction = 0.8

# Get train split index for all dataframes
train_split = [int(split_fraction * len(df)) for df in data_by_library.values()]

In [25]:
### REFERENCE THE CODE HERE IF REVERTING TO AUTOMATIC DATASET CREATION
# # in the end, we only need the percentage data for the model
# # because the order of the data, sorted chronologically, already encodes the time dependency
# data = df['Percentage'].values

# x_train = data[: train_split]
# y_train = data[past : train_split + future]

# x_val = data[train_split : len(data) - future] # don't go to the end, let the future data be the target
# y_val = data[train_split + past :] # offset train_split by future

In [26]:
# Train and validation data containers
train_sequences = []
val_sequences = []
train_library_inputs = []
val_library_inputs = []
train_targets = []
val_targets = []

# Process each library
for (library_key, data), train_idx in zip(data_by_library.items(), train_split):

    # Split into training and validation sets
    occupancy = data['Occupancy'].values
    train_values = occupancy[:train_idx]
    val_values = occupancy[train_idx:]

    # Generate training sequences
    for i in range(0, len(train_values) - sequence_length - future, sampling_rate):
        train_sequences.append(train_values[i:i + sequence_length])
        train_library_inputs.append(library_key)
        train_targets.append(train_values[i + sequence_length + future - 1])

    # Generate validation sequences
    for i in range(0, len(val_values) - sequence_length - future, sampling_rate):
        val_sequences.append(val_values[i:i + sequence_length])
        val_library_inputs.append(library_key)
        val_targets.append(val_values[i + sequence_length + future - 1])

# Convert lists to NumPy arrays
train_sequences = np.array(train_sequences).reshape(-1, sequence_length, 1)
val_sequences = np.array(val_sequences).reshape(-1, sequence_length, 1)
train_library_inputs = np.array(train_library_inputs)
val_library_inputs = np.array(val_library_inputs)
train_targets = np.array(train_targets)
val_targets = np.array(val_targets)

### Build the multi-input model

In [27]:
# Define multi-input GRU model
def build_multi_input_model(sequence_length, num_libraries):
    seq_input = Input(shape=(sequence_length, 1), name="sequence_input")

    x = GRU(128, activation="tanh", return_sequences=True)(seq_input)
    x = Dropout(0.2)(x)
    x = GRU(64, activation="tanh", return_sequences=False)(x)

    lib_input = Input(shape=(num_libraries,), name="library_input")
    combined = Concatenate()([x, lib_input])

    x = Dense(32, activation="relu")(combined)
    output = Dense(1, name="output")(x)

    model = Model(inputs=[seq_input, lib_input], outputs=output)
    model.compile(optimizer="adam", loss="mse", metrics=["mae"])
    return model

# Build the model
sequence_length = 120  # Past timesteps (= 720 / 6)
num_libraries = len(encoder.categories_[0])  # Number of unique libraries

model = build_multi_input_model(sequence_length, num_libraries)


In [28]:
# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience = 2)

learning_rate_reduction = ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001
)

### Train and validate

In [29]:
# RNNs usually slow, computationally expensive:
# calculations are done sequentially, it all depends on the previous output => no parallelization possible

model.fit(
    [train_sequences, train_library_inputs],
    train_targets,
    validation_data=([val_sequences, val_library_inputs], val_targets),
    epochs=3,
    batch_size=64,
    callbacks=[early_stopping, learning_rate_reduction]
)


Epoch 1/3
285/285 ━━━━━━━━━━━━━━━━━━━━ 96s 282ms/step - loss: 0.0183 - mae: 0.0808 - val_loss: 0.0169 - val_mae: 0.0747 - learning_rate: 0.0010
Epoch 2/3
285/285 ━━━━━━━━━━━━━━━━━━━━ 73s 257ms/step - loss: 0.0060 - mae: 0.0457 - val_loss: 0.0146 - val_mae: 0.0618 - learning_rate: 0.0010
Epoch 3/3
285/285 ━━━━━━━━━━━━━━━━━━━━ 67s 236ms/step - loss: 0.0055 - mae: 0.0410 - val_loss: 0.0116 - val_mae: 0.0582 - learning_rate: 0.0010


In [30]:
# Get current and peak memory usage
current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage: {current / 1024 ** 2:.2f} MB")
print(f"Peak memory usage: {peak / 1024 ** 2:.2f} MB")

# Stop tracing
tracemalloc.stop()

Current memory usage: 157.72 MB
Peak memory usage: 170.40 MB
